# 工作任务查询的实现原型

基本思路：

- 使用 LlamaIndex
- 基于 LlamaIndex 的路由查询引擎（`RouterQueryEngine`）
- 实现2个不同的索引，一个用于总结工作任务，一个用于查询具体的任务细节

## 加载数据

In [1]:
%%time

# 数据被简化为一个字符串列表，模拟当天完成的工作任务记录
items=[
"开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引。通过摘要索引，为用户生成的总结性能有大幅度提升。经测试评估，准确率达到了95%，以前是45%。",
"开发部陈逸飞，simple-rag 产品，修正bug：中文断句不完整。之前断句不完整，造成嵌入相似度准确率不高。目前的做法是更换splitter实现，并且对分块参数做了优化。",
"产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询。用户反映以及我们测试，在某些功能下，大文档的检索召回率不高，造成回答效果不好。希望通过增加新的检索方法，并融合现在的检索结果，提高检索召回率。",
"产品部林晓琪，江南皮革厂客服助手项目，和客户开会，沟通需求。确定了客服助手的基本功能，并约定了下一次沟通时间和沟通内容。",
"运维部李泽言，江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题。服务卡的主要原因是算力不足，原来是2张a100 40G，目前扩到了4张。今天业务高峰时间负载不到60%，问题解决。",
"销售部韩雪薇，天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案。资质文案主要包括： 相关软件著作权证书、银行资质证明、国安资质证书和验资报告等，已经整理成word文件。",
"法务部张峻宁，完成审核了与深圳做大事公司的商务合同。之前对方提出了多个版权方面的条款，经过沟通，以及有关方面的查询，修改了相关条款，消除了商业风险，和客户达成了共识。目前已经提交给办公室。",
"开发部郑千，simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误。bug的现象是部分字体显示不全，更换实现库，解决了这个问题。",    
]

CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 8.82 μs


In [2]:
from llama_index.core.schema import TextNode

# 基于数据直接生成为节点

nodes=[ TextNode(text=item, id_=f"node_{index}") for index, item in enumerate(items)]
nodes[0]

TextNode(id_='node_0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引。通过摘要索引，为用户生成的总结性能有大幅度提升。经测试评估，准确率达到了95%，以前是45%。', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

## 摘要索引

In [3]:
%%time

from llama_index.core import SummaryIndex

# 创建摘要索引

summary_index = SummaryIndex(nodes)

CPU times: user 259 ms, sys: 19.4 ms, total: 279 ms
Wall time: 278 ms


In [4]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike

# 创建全局llm，查询需要用到

Settings.llm=OpenAILike(
    # model="llama3.1",  # 工作总结不能正确识别示例说明
    # model="qwen2", # json parse 报错；生成内容不稳定，有时遗漏
    # model="qwen-turbo", # 正确运行
    # model="glm4", # 正确运行，但不是很稳定
    # model="yi:9b", # 生成的内容错误，和示例混合
    # model="yi-34b-chat", # 能运行，有问题，一个任务描述摘要会拆分成好几个条目
    # model="gpt-3.5-turbo", # 正确运行
    # model="glm-3-turbo", # 正确运行
    # model="glm-4", # 正确运行
    # model="glm-4-air", # 正确运行
    # model="Baichuan2-Turbo", # 有不符合要求的情况 - 1个工作任务拆分成多条
    # model="Baichuan3-Turbo", # 正确运行
    # model="Baichuan4", # 有遗漏总结内容的情况
    # model="ERNIE-Speed-8K", # 生成内容解析报错
    # model="ERNIE-3.5-8K-0205", # 生成内容解析报错
    # model="ERNIE-4.0-8K", # 正确运行
    # model="moonshot-v1-8k", # 不行，做摘要有任务被拆分为2个，摘要不稳定，经常负载饱和
    # model="abab5.5s-chat", # 报错，生成内容解析错
    # model="abab6.5s-chat", # 报错，生成内容解析错
    # model="qwen:14b", # 不行，摘要内容不符合要求
    model="qwen2-72b-instruct", # 报错，生成内容解析错，但生成摘要内容很稳定
    # model="hunyuan-lite", # 不行，生成的内容错误，和示例混合
    # model="hunyuan-standard", # 不行，生成摘要有遗漏，生成内容解析报错
    # model="hunyuan-pro", # 正确运行
    # model="deepseek-v2", # 不行，摘要稳定的遗漏内容并混杂示例内容
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0,
    request_timeout=60.0
)

CPU times: user 1.55 s, sys: 188 ms, total: 1.73 s
Wall time: 1.73 s


In [5]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# 创建摘要查询引擎

summary_retriever=summary_index.as_retriever()

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

summary_query_engine = RetrieverQueryEngine(
    retriever=summary_retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 165 μs, sys: 0 ns, total: 165 μs
Wall time: 169 μs


In [6]:
%%time

# 通过摘要查询引擎生成工作摘要

response = summary_query_engine.query(
    "给出涉及的所有部门所有人的任务摘要输出，输出示例如下："
    """
    产品部:

    - 张三
        - xxx产品，提交新特性需求: xxx
        - xxx项目，和客户开会，xxxxx
        - ...
    - 李四
        - xxxx
        - ...
    - 王五
      - ...

    开发部:
    
    - 赵六
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    - 于七
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    ...

    xxx部:
    ...
    """
)

response.print_response_stream()
print()

产品部:

- 林晓琪
    - simple-rag 产品，提交新特性需求: 多数据源的融合检索查询
    - 江南皮革厂客服助手项目，和客户开会，沟通需求

开发部:

- 韩宇轩
    - simple-rag 产品，实现新特性: 针对文档的摘要索引
- 陈逸飞
    - simple-rag 产品，修正bug: 中文断句不完整
- 郑千
    - simple-rag 产品，修正bug: 聊天界面 markdown 格式显示错误

运维部:

- 李泽言
    - 江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题

销售部:

- 韩雪薇
    - 天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案

法务部:

- 张峻宁
    - 完成审核了与深圳做大事公司的商务合同，消除了商业风险，和客户达成了共识
CPU times: user 180 ms, sys: 29.1 ms, total: 209 ms
Wall time: 9.69 s


In [7]:
%%time

response = summary_query_engine.query("林晓琪完成哪些任务，列表说明")
response.print_response_stream()
print()

林晓琪完成了以下任务：
- 提交了simple-rag产品的多数据源融合检索查询的新特性需求，旨在提高大文档的检索召回率，以改善回答效果。
- 参与了江南皮革厂客服助手项目的客户需求沟通会议，确认了客服助手的基本功能，并安排了下次会议的时间和内容。
CPU times: user 54.7 ms, sys: 4.4 ms, total: 59.2 ms
Wall time: 2.87 s


## 向量索引

In [8]:
%%time

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

# 设置嵌入模型

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    # model_name="bge-m3",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 413 ms, sys: 36.2 ms, total: 449 ms
Wall time: 448 ms


In [9]:
%%time

from llama_index.core import VectorStoreIndex

# 创建向量索引

vector_index = VectorStoreIndex(nodes)

CPU times: user 49 ms, sys: 3.27 ms, total: 52.3 ms
Wall time: 817 ms


In [10]:
%%time

# 创建向量查询引擎

vector_retriever=vector_index.as_retriever()

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 175 μs, sys: 0 ns, total: 175 μs
Wall time: 178 μs


In [11]:
%%time

vector_retriever.retrieve("为什么要做多数据源的融合检索查询？是谁提出的？")

CPU times: user 6.64 ms, sys: 0 ns, total: 6.64 ms
Wall time: 62.3 ms


[NodeWithScore(node=TextNode(id_='node_2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询。用户反映以及我们测试，在某些功能下，大文档的检索召回率不高，造成回答效果不好。希望通过增加新的检索方法，并融合现在的检索结果，提高检索召回率。', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.5919122817844211),
 NodeWithScore(node=TextNode(id_='node_0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引。通过摘要索引，为用户生成的总结性能有大幅度提升。经测试评估，准确率达到了95%，以前是45%。', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.3938835184407688)]

In [12]:
%%time

response = vector_query_engine.query("为什么要做多数据源的融合检索查询？是谁提出的")
response.print_response_stream()
print()

产品部的林晓琪提出了多数据源的融合检索查询的新特性需求。原因是用户反馈和内部测试显示，在某些功能下，大文档的检索召回率不高，影响了回答效果。通过增加新的检索方法并融合现有结果，旨在提高检索召回率。
CPU times: user 58.6 ms, sys: 886 μs, total: 59.4 ms
Wall time: 5.15 s


## 路由查询引擎

In [13]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.52 ms, sys: 209 μs, total: 1.73 ms
Wall time: 1.4 ms


In [14]:
%%time

from llama_index.core.tools import QueryEngineTool

# 创建查询相关的 tool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "对工作任务总结方面的问题很有用。"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于回答工作任务相关的具体问题。"
    ),
)

CPU times: user 32 μs, sys: 4 μs, total: 36 μs
Wall time: 40.1 μs


In [15]:
%%time

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# 创建路由查询引擎

from llama_index.core.response_synthesizers import TreeSummarize

summarizer = TreeSummarize(streaming=True, use_async=False)

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(
        llm=OpenAILike(
            # model="qwen-turbo", # 正确运行
            model="glm4", # 正确运行，但不是很稳定
            api_base="http://ape:3000/v1", 
            api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
            is_chat_model=True,
            temperature=0,
            request_timeout=60.0
        )
    ),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    summarizer=summarizer
)

CPU times: user 371 μs, sys: 0 ns, total: 371 μs
Wall time: 376 μs


### 总结性的问题

In [16]:
%%time

response = query_engine.query(
    "给出涉及的所有部门所有人的任务摘要输出，输出示例如下："
    """
    产品部:

    - 张三
        - xxx产品，提交新特性需求: xxx
        - xxx项目，和客户开会，xxxxx
        - ...
    - 李四
        - xxxx
        - ...
    - 王五
      - ...

    开发部:
    
    - 赵六
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    - 于七
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    ...

    xxx部:
    ...
    """
)

response.print_response_stream()
print()

产品部:

- 林晓琪
    - simple-rag 产品，提交新特性需求: 多数据源的融合检索查询
    - 江南皮革厂客服助手项目，和客户开会，沟通需求

开发部:

- 韩宇轩
    - simple-rag 产品，实现新特性: 针对文档的摘要索引
- 陈逸飞
    - simple-rag 产品，修正bug: 中文断句不完整
- 郑千
    - simple-rag 产品，修正bug: 聊天界面 markdown 格式显示错误

运维部:

- 李泽言
    - 江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题

销售部:

- 韩雪薇
    - 天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案

法务部:

- 张峻宁
    - 完成审核了与深圳做大事公司的商务合同，消除了商业风险，和客户达成了共识
CPU times: user 175 ms, sys: 44.8 ms, total: 220 ms
Wall time: 18.4 s


In [17]:
%%time

response = query_engine.query(
    "列表简要回答，林晓琪做了哪些工作任务"
    "这是总结性质的问题"
)

response.print_response_stream()
print()

林晓琪负责的两项主要任务包括：
1. 针对simple-rag产品提出新特性需求，旨在通过融合多数据源的检索查询来提升大文档的检索召回率，以改善回答效果。
2. 参与江南皮革厂客服助手项目的客户需求沟通会议，明确了客服助手的基本功能，并安排了后续沟通计划。
CPU times: user 57.8 ms, sys: 9.14 ms, total: 66.9 ms
Wall time: 5.29 s


### 具体问题

In [18]:
%%time

response = query_engine.query("为什么要做多数据源的融合检索查询？是谁提出的")

response.print_response_stream()
print()

产品部的林晓琪提出了多数据源的融合检索查询的新特性需求。原因是用户反馈和内部测试显示，在某些功能下，大文档的检索召回率不高，影响了回答效果。通过增加新的检索方法并融合现有结果，旨在提高检索召回率。
CPU times: user 51.2 ms, sys: 13.6 ms, total: 64.8 ms
Wall time: 4.38 s
